In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import torch
from torch import nn

In [ ]:
import kagglehub

In [ ]:
anime_path = kagglehub.dataset_download(handle='splcher/animefacedataset')

In [ ]:
anime_path

In [ ]:
from torchvision import transforms as T
from torchvision.datasets import ImageFolder

In [ ]:
transform = T.Compose([
    T.Resize((64, 64)),
    T.ToTensor(),
    T.Normalize([.5, .5, .5], [.5, .5, .5]),
])

In [ ]:
train_data = ImageFolder(root=anime_path, transform=transform)

In [ ]:
train_data

In [ ]:
batch_size = 128

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

In [ ]:
train_loader

In [ ]:
image0, _ = train_data[0]

In [ ]:
image0.shape

In [ ]:
plt.imshow(image0.permute(1, 2, 0)*.5 + .5)
plt.show()

In [ ]:
def plot_images(imgs):
    for i in range(32):
        ax = plt.subplot(4, 8, i + 1)
        plt.imshow(imgs[i].permute(1,2,0)/2+0.5)
        plt.xticks([])
        plt.yticks([])
    plt.subplots_adjust(hspace=-0.6)
    plt.show()

In [ ]:
imgs, _ = next(iter(train_loader))

In [ ]:
plot_images(imgs)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

In [ ]:
device

In [ ]:
D = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.LeakyReLU(.2, inplace=True),
    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(num_features=128),
    nn.LeakyReLU(.2, inplace=True),
    nn.Conv2d(in_channels=128, out_channels=256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(num_features=256),
    nn.LeakyReLU(.2, inplace=True),
    nn.Conv2d(in_channels=256, out_channels=512, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(num_features=512),
    nn.LeakyReLU(.2, inplace=True),
    nn.Conv2d(in_channels=512, out_channels=1, kernel_size=4, stride=1, padding=0, bias=False),
    nn.Sigmoid(),
    nn.Flatten(),
).to(device)

In [ ]:
G = nn.Sequential(
    nn.ConvTranspose2d(in_channels=100, out_channels=512, kernel_size=4, stride=1, padding=0, bias=False),
    nn.BatchNorm2d(512),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=4, stride=2, padding=1, bias=False),
    nn.Tanh()
).to(device)

In [ ]:
sample_noise = torch.randn(32, 100, 1, 1)

In [ ]:
loss_fn = nn.BCELoss()
lr = .0002
optimG = torch.optim.Adam(G.parameters(), lr=lr, betas=(.5, .999))
optimD = torch.optim.Adam(D.parameters(), lr=lr, betas=(.5, .999))


In [ ]:
def test_epoch():
    noise=torch.randn(32,100,1,1).to(device=device)
    fake_samples=G(noise).cpu().detach()
    for i in range(32):
        ax = plt.subplot(4, 8, i + 1)
        img=(fake_samples.cpu().detach()[i]/2+0.5).permute(1,2,0)
        plt.imshow(img)
        plt.xticks([])
        plt.yticks([])
    plt.subplots_adjust(hspace=-0.6)
    plt.show()

In [ ]:
test_epoch()

In [ ]:
def train_D_on_real(real_samples):
    optimD.zero_grad()
    output = D(real_samples.to(device))
    loss = loss_fn(output, torch.ones_like(output, dtype=torch.float32))
    loss.backward() 
    optimD.step()
    return loss


In [ ]:
def train_D_on_fake():
    optimD.zero_grad()
    fake_samples = G(torch.randn(batch_size, 100, 1, 1).to(device))
    output = D(fake_samples)
    loss = loss_fn(output, torch.zeros_like(output, dtype=torch.float32))
    loss.backward()
    optimD.step()
    return loss

In [ ]:
def train_G():
    optimG.zero_grad()
    generated_result = G(torch.randn(batch_size, 100, 1, 1).to(device))
    output = D(generated_result)
    loss = loss_fn(output, torch.ones_like(output, dtype=torch.float32))
    loss.backward()
    optimG.step()
    return loss

In [ ]:
def train(epochs=20):
    for i in range(epochs):
        gloss, dloss = 0, 0
        for n, (real_samples, _) in enumerate(train_loader):
            loss_D = train_D_on_real(real_samples)
            dloss += loss_D
            loss_D = train_D_on_fake()
            dloss += loss_D
            gloss += train_G()
        gloss /= n
        dloss /= n
        print(f"epoch {i+1}, dloss: {dloss}, gloss {gloss}")
        test_epoch()


In [ ]:
train()